# liboqs_2_csv_merge
Dieses Notebook verarbeitet die vier *liboqs*-CSV-Dateien aus dem Verzeichnis `output/csv` und führt sie in einer einheitlichen Struktur zusammen.

In [3]:
import pandas as pd
import numpy as np

# 1. Verzeichnis mit den CSV-Dateien und Ausgabedatei
csv_dir = 'output/csv'
output_file = f"{csv_dir}/liboqs_all.csv"

# 2. CSV-Dateien laden
mem_kem = pd.read_csv(f"{csv_dir}/liboqs_mem_kem.csv")
mem_sig = pd.read_csv(f"{csv_dir}/liboqs_mem_sig.csv")
time_kem = pd.read_csv(f"{csv_dir}/liboqs_time_kem.csv")
time_sig = pd.read_csv(f"{csv_dir}/liboqs_time_sig.csv")

# Neue Dateien mit vollständigem Heap
mem_kem_full = pd.read_csv(f"{csv_dir}/liboqs_mem_kem_fullHeap.csv")
mem_sig_full = pd.read_csv(f"{csv_dir}/liboqs_mem_sig_fullHeap.csv")

# 3. Zeiten von Millisekunden in Mikrosekunden umrechnen
time_kem['keygen_time'] = time_kem['kem_keypair (ms)'] * 1000
time_kem['encaps_time'] = time_kem['kem_enc (ms)']     * 1000
time_kem['decaps_time'] = time_kem['kem_dec (ms)']     * 1000

time_sig['keygen_time'] = time_sig['keypair (ms)']     * 1000
time_sig['encaps_time'] = time_sig['sign (ms)']        * 1000
time_sig['decaps_time'] = time_sig['verify (ms)']      * 1000

# 4. Zeitdaten extrahieren und "Typ"-Spalte hinzufügen
df_time_kem = time_kem.loc[:, ['Model', 'Algorithmus', 'keygen_time', 'encaps_time', 'decaps_time']].copy()
df_time_kem.insert(2, 'Typ', 'kem')

df_time_sig = time_sig.loc[:, ['Model', 'Algorithmus', 'keygen_time', 'encaps_time', 'decaps_time']].copy()
df_time_sig.insert(2, 'Typ', 'sig')

# 5. Speicherverbrauchsdaten zusammenführen
mem_kem.insert(2, 'Typ', 'kem')
mem_sig.insert(2, 'Typ', 'sig')
mem = pd.concat([mem_kem, mem_sig], ignore_index=True)
mem = mem.loc[:, ['Model', 'Algorithmus', 'Typ',
                  'maxStack_key', 'maxStack_enc', 'maxStack_dec']].copy()
mem.rename(columns={
    'maxStack_key': 'stack_key',
    'maxStack_enc': 'stack_enc',
    'maxStack_dec': 'stack_dec'
}, inplace=True)

mem_kem_full.insert(2, 'Typ', 'kem')
mem_sig_full.insert(2, 'Typ', 'sig')
mem_heap = pd.concat([mem_kem_full, mem_sig_full], ignore_index=True)
mem_heap = mem_heap.loc[:, ['Model', 'Algorithmus', 'Typ',
                            'maxHeap_key', 'maxHeap_enc', 'maxHeap_dec']].copy()
mem_heap.rename(columns={
    'maxHeap_key':  'heap_key',
    'maxHeap_enc':  'heap_enc',
    'maxHeap_dec':  'heap_dec'
}, inplace=True)

# 6. Kombinieren von Heap- und Stack-Daten (inkl. Typ)
mem_combined = pd.merge(mem_heap, mem, on=['Model', 'Algorithmus', 'Typ'], how='left')

# 7. Zeitdaten zusammenführen
df_time = pd.concat([df_time_kem, df_time_sig], ignore_index=True)

# 8. Merge Zeit + Speicher (inkl. Typ)
df_merged = pd.merge(df_time, mem_combined, on=['Model', 'Algorithmus', 'Typ'], how='left')

# 9. Spalten neu anordnen
column_order = [
    'Model', 'Algorithmus', 'Typ',
    'keygen_time', 'encaps_time', 'decaps_time',
    'heap_key', 'heap_enc', 'heap_dec',
    'stack_key', 'stack_enc', 'stack_dec'
]
df_merged = df_merged[column_order]

# 10. Zeitspalten runden
time_cols = ['keygen_time', 'encaps_time', 'decaps_time']
for col in time_cols:
    df_merged[col] = df_merged[col].round().astype(int)

# 11. Speicherwerte runden
mem_cols = [c for c in df_merged.columns if c not in ['Model', 'Algorithmus', 'Typ'] + time_cols]
for col in mem_cols:
    if pd.api.types.is_numeric_dtype(df_merged[col]):
        df_merged[col] = df_merged[col].round(1)

# 12. Ergebnis speichern
df_merged.to_csv(output_file, index=False)
print(f"Ausgabedatei erstellt: {output_file}")


Ausgabedatei erstellt: output/csv/liboqs_all.csv
